In [26]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)

In [28]:
homePath = 'game'
trainPath = os.path.join(homePath, 'train_n.csv')
trainData = pd.read_csv(trainPath)
testPath = os.path.join(homePath, 'test_n.csv')
testData = pd.read_csv(testPath)

In [29]:
trainLabel = trainData['acc_now_delinq']
testLabel = testData['acc_now_delinq']
del trainData['acc_now_delinq']
del trainData['member_id']
del trainData['index']
del testData['acc_now_delinq']
del testData['member_id']
del testData['index']

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, n_jobs=-1)
scores = cross_val_score(rnd_clf, trainData, trainLabel,
                         scoring="recall", cv=10)
scores.mean()

0.78469650916459421

In [43]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
etr_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=50, n_jobs=-1)
# etr_score = cross_val_score(etr_clf, trainData, trainLabel, scoring='recall', cv=10)
# etr_score.mean()

In [44]:
etr_clf.fit(trainData, trainLabel)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=50,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [60]:
from sklearn.metrics import precision_score, recall_score
def calc_f2(label, predict):
    p = precision_score(label, predict)
    r = recall_score(label, predict)
    f2_score = 5*p*r / (4*p + r)
    return f2_score

In [61]:
y_predictions = etr_clf.predict(trainData)
f2 = calc_f2(trainLabel, y_predictions)
f2

0.79141177169329457

In [65]:
y_test = etr_clf.predict(testData)


In [68]:
calc_f2(testLabel, y_test)

E:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
E:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


nan

In [67]:
y_predictions

array([ 1.,  1.,  1., ...,  0.,  0.,  1.])